<a href="https://colab.research.google.com/github/tre787/VisualDataScience/blob/main/Visual_Data_Science_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Visual Data Science Report**
Trevor Calvin Baretto - 12332204

Link for the colab notebook - https://colab.research.google.com/drive/1bAzTjlu5IfpsEZ6qC2o08NrvcQlCwMH6?usp=sharing

In [2]:
# @title
!pip install dash
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Load datasets
co2_emissions = pd.read_csv('Carbon_(CO2)_Emissions_by_Country.csv', encoding='utf-8')
temp_changes = pd.read_csv('Environment_Temperature_change_E_All_Data_NOFLAG.csv', encoding='ISO-8859-1')

# Clean CO₂ Emissions Data
co2_emissions.rename(columns={'Date': 'Year'}, inplace=True)
co2_emissions['Year'] = pd.to_datetime(co2_emissions['Year'], errors='coerce').dt.year
co2_emissions.dropna(subset=['Year'], inplace=True)
co2_emissions['Year'] = co2_emissions['Year'].astype(int)

# Ensure Unique Country-Year Values for CO₂
co2_emissions = co2_emissions.groupby(['Country', 'Year'])['Kilotons of Co2'].mean().reset_index()

# Reshape and Clean Temperature Dataset
temp_changes_long = temp_changes.melt(
    id_vars=['Area'],
    value_vars=[col for col in temp_changes.columns if col.startswith('Y')],
    var_name='Year',
    value_name='Temperature_Change'
)

temp_changes_long.rename(columns={'Area': 'Country'}, inplace=True)
temp_changes_long['Year'] = temp_changes_long['Year'].str[1:].astype(int)

# Ensure Unique Country-Year Values for Temperature Change
temp_changes_long = temp_changes_long.groupby(['Country', 'Year'])['Temperature_Change'].mean().reset_index()

# Standardize Country Names
co2_emissions['Country'] = co2_emissions['Country'].str.strip().str.lower()
temp_changes_long['Country'] = temp_changes_long['Country'].str.strip().str.lower()

# Merge Cleaned Datasets
data = co2_emissions.merge(temp_changes_long, on=['Country', 'Year'], how='inner')

# Check for Duplicates
data = data.drop_duplicates(subset=['Country', 'Year'])

# Convert country names to title case
data['Country'] = data['Country'].str.title()

# Get min and max years for the slider
min_year = data['Year'].min()
max_year = data['Year'].max()

# Initialize Dash App
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("CO2 Emissions and Temperature Changes Dashboard"),

    # Dropdown for selecting country
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country.title(), 'value': country} for country in data['Country'].unique()],
        value='Austria',
        multi=False
    ),

    dcc.Graph(id='co2-trend'),
    dcc.Graph(id='temp-trend'),
    dcc.Graph(id='scatter-co2-temp'),

    # Slider for selecting year
    html.Label("Select Year for Choropleth Maps and Bar Chart:"),
    dcc.Slider(
        id='year-slider',
        min=min_year,
        max=max_year,
        value=2010,  # Default year
        marks={str(year): str(year) for year in range(min_year, max_year+1, 5)},  # Show marks every 5 years
        step=1
    ),

    # Choropleth Maps
    dcc.Graph(id='co2-choropleth'),
    dcc.Graph(id='temp-choropleth'),

    # Bar Chart: Top CO₂ Emitting Countries
    dcc.Graph(id='top-co2-bar')
])

@app.callback(
    [Output('co2-trend', 'figure'),
     Output('temp-trend', 'figure'),
     Output('scatter-co2-temp', 'figure')],
    [Input('country-dropdown', 'value')]
)
def update_country_graphs(selected_country):
    filtered_data = data[data['Country'] == selected_country]

    if filtered_data.empty:
        return px.scatter(title="No Data Available"), px.line(title="No Data Available"), px.line(title="No Data Available")

    co2_fig = px.line(
        filtered_data.sort_values(by='Year'),
        x='Year', y='Kilotons of Co2',
        title=f'CO2 Emissions Over Time - {selected_country.title()}'
    )

    temp_fig = px.line(
        filtered_data.sort_values(by='Year'),
        x='Year', y='Temperature_Change',
        title=f'Temperature Changes Over Time - {selected_country.title()}'
    )

    scatter_fig = px.scatter(
        filtered_data,
        x='Kilotons of Co2', y='Temperature_Change',
        title=f'CO2 Emissions vs Temperature Change - {selected_country.title()}',
        trendline='ols'
    )

    return co2_fig, temp_fig, scatter_fig


@app.callback(
    [Output('co2-choropleth', 'figure'),
     Output('temp-choropleth', 'figure'),
     Output('top-co2-bar', 'figure')],
    [Input('year-slider', 'value')]
)
def update_choropleth_and_bar(selected_year):
    yearly_data = data[data['Year'] == selected_year]

    if yearly_data.empty:
        return px.choropleth(title="No Data Available"), px.choropleth(title="No Data Available"), px.bar(title="No Data Available")

    # Choropleth Map for CO2 Emissions
    co2_choropleth = px.choropleth(
        yearly_data,
        locations="Country",
        locationmode='country names',
        color="Kilotons of Co2",
        hover_name="Country",
        color_continuous_scale=px.colors.sequential.Plasma,
        title=f"CO2 Emissions by Country in {selected_year}"
    )

    # Choropleth Map for Temperature Change
    temp_choropleth = px.choropleth(
        yearly_data,
        locations="Country",
        locationmode='country names',
        color="Temperature_Change",
        hover_name="Country",
        color_continuous_scale=px.colors.sequential.Viridis,
        title=f"Temperature Change by Country in {selected_year}"
    )

    # Bar Chart: Top CO₂ Emitting Countries
    top_countries = yearly_data.nlargest(10, 'Kilotons of Co2')  # Get top 10 emitters
    co2_bar_chart = px.bar(
        top_countries,
        x='Kilotons of Co2',
        y='Country',
        orientation='h',
        title=f"Top 10 CO₂ Emitting Countries in {selected_year}",
        labels={'Kilotons of Co2': 'CO₂ Emissions (Kilotons)'},
        color='Kilotons of Co2',  # Add color scaling based on emissions
        color_continuous_scale=px.colors.sequential.Blues
    )

    return co2_choropleth, temp_choropleth, co2_bar_chart

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')